# **Chat Interface**

In [ ]:
!pip install openai
!pip install langchain
!pip -q install huggingface_hub google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 9.1 MB/s eta 0:00:00


In [ ]:
import os
import langchain
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate

from enum import Enum
from pydantic import BaseModel, Field

os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(temperature=0, model="gpt-4o")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
!pip install pydantic

In [ ]:
import pydantic
# Formerly, I capitalized that

In [ ]:
from pydantic import BaseModel, Field

class PersonalDetails(BaseModel):
    labor_type: str = Field(
        ...,
        description="If labor costs should be taken into account and what type of labor is currently used (e.g., long-term contracts, long-term contracts, reliance on overtime)?",
    )
    capacity_constraining_factors: str = Field(
        ...,
        description="What are currently constraining factors in the plant? (workforce, machine availablitiy?...)",
    )
    variable_costs: str = Field(
        ...,
        description="Which variable costs of producing the product are known",
    )
    workload_variation: str = Field(
        ...,
        description="Is the workload subject to strong variations?"
    )
    product_mix: str = Field(
        ...,
        description="Which products are sold and how is their realtionship to one another?"
    )

def check_what_is_empty(user_personal_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_personal_details.dict().items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            ask_for.append(field)
    return ask_for

def ask_for_info(field_name):
    # Create a mapping of field names to human-readable questions
    field_to_question = {
        'labor_type': "Can you please provide the type of labor used (e.g., long-term contracts, reliance on overtime)?",
        'capacity_constraining_factors': "What are currently the constraining factors in the plant (e.g., workforce, machine availability)?",
        'variable_costs': "Which variable costs of producing the product are known?",
        'workload_variation': "Is the workload subject to strong variations?",
        'product_mix': "Which products are sold and how is their relationship to one another?"
    }

    # Generate the question for the specific field
    question = field_to_question.get(field_name, "")
    return question

def update_user_details(user_details, field_name, response):
    # Directly update the field with the response
    setattr(user_details, field_name, response)
    return user_details


In [ ]:
# Initialize user details
user_123_personal_details = PersonalDetails(
    labor_type="",
    capacity_constraining_factors="",
    variable_costs="",
    workload_variation="",
    product_mix=""
)

user_details = user_123_personal_details

while True:
    # Check what information is still needed
    ask_for = check_what_is_empty(user_details)

    # If all information is gathered, exit the loop
    if not ask_for:
        print("All information gathered. Moving to the next phase.")
        break

    # Get the next field that needs to be filled
    next_field = ask_for[0]

    # Generate a conversational question based on the missing information
    ai_response = ask_for_info(next_field)
    print(ai_response)

    # Get user's response
    user_response = input("> ")

    # Process the response and update user details
    user_details = update_user_details(user_details, next_field, user_response)

print("Final user details:", user_details.dict())

Can you please provide the type of labor used (e.g., long-term contracts, reliance on overtime)?
> mix of long term constracts and short term contracts, overtime used as well
What are currently the constraining factors in the plant (e.g., workforce, machine availability)?
> Capaity of the workstations, plant size, labor force size, customer demands and the ability of a supplier to supply pre processed materialso backorders allowed, 
Which variable costs of producing the product are known?
> raw materials, electricity, holding costs
Is the workload subject to strong variations?
> Yes
Which products are sold and how is their relationship to one another?
> One main product and multiple products of minor importance (minor sales volume)
All information gathered. Moving to the next phase.
Final user details: {'labor_type': 'mix of long term constracts and short term contracts, overtime used as well', 'capacity_constraining_factors': 'Capaity of the workstations, plant size, labor force size,

In [ ]:
user_details

PersonalDetails(labor_type='mix of long term constracts and short term contracts, overtime used as well', capacity_constraining_factors='Capaity of the workstations, plant size, labor force size, customer demands and the ability of a supplier to supply pre processed materialso backorders allowed, ', variable_costs='raw materials, electricity, holding costs', workload_variation='Yes', product_mix='One main product and multiple products of minor importance (minor sales volume)')

# **Decision Making with Factory Physics - RAG**

In [ ]:
!pip install pulp
!pip install python-dotenv
!pip install -U chromadb
!pip install -U tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7

New try with the GPT4 API

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
import os
from dotenv import load_dotenv
import json

In [ ]:
#loader = TextLoader(file_path="../data/PaulGrahamEssays/vb.txt")

## Other options for loaders
# loader = PyPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [ ]:
import json

# Define the path to your file
file_path = 'train (2).jsonl'

# Define a simple Document class to hold content and metadata
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata if metadata else {}

# Read the JSONL file and convert chunks to Document objects
documents = []
with open(file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        chunk = data['chunk']
        documents.append(Document(page_content=chunk))

# Check the number of documents loaded
print(f'Loaded {len(documents)} documents.')

Loaded 50 documents.


In [ ]:
import openai
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
import chromadb
import tiktoken
# Load data into Chroma vector store
vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
# Define your query
query = f"Which basic model and which extension(s) of the basic model should I use. You must return one or more of the 8 models, while combinations are also possible: Which production planning model from the book factory physics would be the most appropriate and why? The possible answers are: Aggregate Planning (AP), Single Product, I. Basic model; Aggregate Planning (AP), Product Mix Planning II. Basic model; Aggregate Planning (AP), Product Mix Planning III. Capacitated ressources; Aggregate Planning (AP), Product Mix Planning IV. Utilization Matching; Aggregate Planning (AP), Product Mix Planning V. Backorders; Aggregate Planning (AP), Product Mix Planning VI. Overtime; Aggregate Planning (AP), Product Mix Planning VII. Yield loss; Workforce Planning (WP), VIII. Basic model. The company's envorinment is as follows: {user_details}"

# Perform similarity search
docs = vectorstore.similarity_search(query, k=10)

In [ ]:
# Initialize the LLM and QA chain using GPT-4
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
# Run the query through the chain
answer = chain.run(input_documents=docs, question=query)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Given the company's environment, the most appropriate production planning models from the book "Factory Physics" would be:

1. **Aggregate Planning (AP), Product Mix Planning II. Basic model**: This model is suitable because the company deals with one main product and multiple minor products. The basic product mix planning model will help in managing the production of multiple products explicitly.

2. **Aggregate Planning (AP), Product Mix Planning III. Capacitated resources**: Since the capacity of workstations, plant size, and labor force size are constraining factors, this model will help in managing these constraints effectively.

3. **Aggregate Planning (AP), Product Mix Planning VI. Overtime**: Given that the company uses overtime, this model will help in planning and optimizing the use of overtime to meet production demands.

4. **Workforce Planning (WP), VIII. Basic model**: This model is essential because the company has a mix of long-term and short-term contracts and uses ove

In [ ]:
answer

NameError: name 'answer' is not defined